In [2]:
### PLEASE COPY NOTEBOOKS TO YOUR FOLDERS TO PREVENT COMMIT CONFLICTS
from dcicutils import ff_utils
from functions.notebook_functions import *
import json
import glob
import os

source_env = 'fourfront-hotseat'
source_key = ff_utils.get_authentication_with_server({}, ff_env=source_env)

find_linked = ['311d0f4f-56ee-4450-8cbb-780c10229284']
store = get_query_or_linked(source_key, linked=find_linked, linked_frame='raw')

print(store.keys())
print(len([i['uuid'] for key in store for i in store[key]]))

store['page'] = [i for i in store['page'] if i['uuid'] not in find_linked]
print(len([i['uuid'] for key in store for i in store[key]]))

#reomve award lab user
store.pop('award')
store.pop('lab')
store.pop('user')
print(len([i['uuid'] for key in store for i in store[key]]))

for i in store:
    print(i, len(store[i]))

dict_keys(['lab', 'user', 'static_section', 'page', 'award'])
110
109
104
static_section 72
page 32


In [5]:
only_post = False
check_only = False

transfer_env = 'data'
transfer_key = ff_utils.get_authentication_with_server({}, ff_env=transfer_env)
schema_name = get_schema_names(transfer_key) 

# reverse lookup dictionary for schema names
rev_schema_name = {}
for key, name in schema_name.items():
    rev_schema_name[name] = schema_name[key]

my_types = [i for i in ORDER if i in store.keys()]
second_round_items = {}

# Round I - only put the required - skip if exists already
for a_type in my_types:
    posted = 0
    print(a_type)
    obj_type = rev_schema_name[a_type]
    # find required field
    schema_info = ff_utils.get_metadata('/profiles/{}.json'.format(a_type), key=transfer_key)
    req_fields = schema_info['required']
    ids = schema_info['identifyingProperties']
    first_fields = list(set(req_fields+ids))
    remove_existing_items = []
    counter=0
    print(len(store[a_type]), 'items exist on source')
    for an_item in store[a_type]:
        counter += 1
        #print(counter, an_item['uuid'])
        # does the item exist
        exists = False
        try:
            # TODO check with all identifiers
            existing = ff_utils.get_metadata(an_item['uuid'], key=transfer_key)
            exists = True
        except:
            exists = False
        if only_post:
            # skip the items that exists
            if exists and existing:
                remove_existing_items.append(an_item['uuid'])
                print("{} {} can not post item".format(obj_type, an_item['uuid']))
                continue
        if not exists:
            posted += 1
            post_first = {key:value for (key,value) in an_item.items() if key in first_fields}
            if check_only:
                ff_utils.post_metadata(post_first, obj_type, key = transfer_key, add_on='check_only=True')            
            else:
                ff_utils.post_metadata(post_first, obj_type, key = transfer_key)
   
    second_round_items[a_type] = [i for i in store[a_type] if i['uuid'] not in remove_existing_items]
    print(posted, 'items posted to target')
    print(len(second_round_items[a_type]), 'items will be patched in second round')
    print()

    

static_section
72 items exist on source
32 items posted to target
72 items will be patched in second round

page
32 items exist on source
8 items posted to target
32 items will be patched in second round



In [9]:
check_only = False
# Round II - patch the rest of the metadata
for a_type in my_types:
    obj_type = rev_schema_name[a_type]
    if not second_round_items[a_type]:
        continue 
    for an_item in second_round_items[a_type]:
        counter += 1
        if check_only:
            ff_utils.patch_metadata(an_item, obj_id = an_item['uuid'], key = transfer_key, add_on='check_only=True')           
        else:
             ff_utils.patch_metadata(an_item, obj_id = an_item['uuid'], key = transfer_key)

In [13]:
data_env = 'data'
data_key = ff_utils.get_authentication_with_server({}, ff_env=data_env)

find_linked = ['311d0f4f-56ee-4450-8cbb-780c10229284']
store = get_query_or_linked(data_key, linked=find_linked)

# for k in store:
#     print(k)
#     for i in store[k]:
#         print(i['uuid'], i['status'], i['display_title'])
#     print()
    

In [28]:
start = '311d0f4f-56ee-4450-8cbb-780c10229284'
start_resp = ff_utils.get_metadata(start, key = data_key)
print(start_resp['title'])

def print_content(cont_embed, data_key):
    for cont in cont_embed:
        cont_uuid = cont['uuid']
        cont_dis = cont['display_title']
        cont_resp = ff_utils.get_metadata(cont_uuid, key = data_key)
        lorem = 'No'
        if cont_resp.get('content'):
            if 'lorem ipsum' in cont_resp['content'].lower():
                lorem = 'Yes'
        print(';'.join(["", 'section', cont_dis, cont_resp['status'], data_key['server']+'/'+cont_resp['uuid'], lorem]))
    
def rec_status_check(resp, data_key, n=0):
    
    print(';'.join([str(n), 'PAGE', resp['display_title'], resp['status'], data_key['server']+'/'+resp['uuid'], ""]))
    if resp.get('content'):
            print_content(resp['content'], data_key)
    if not resp.get('children'):
        return
    else:
        for ch in resp['children']:
            ch_resp = ff_utils.get_metadata(ch['uuid'], key = data_key)
            rec_status_check(ch_resp, data_key, n=n+1)
            
rec_status_check(start_resp, data_key)           


Help Section
0;PAGE;Help Section;released;https://data.4dnucleome.org/311d0f4f-56ee-4450-8cbb-780c10229284;
;section;help.user-guide.getting-started.getting-started;released;https://data.4dnucleome.org/442c8aa0-dc6c-43d7-814a-854af460b011;No
;section;help.user-guide.data-model.slidecarousel;released;https://data.4dnucleome.org/0af523d3-5772-4120-898c-0487d51c7b77;No
1;PAGE;User Guide;released;https://data.4dnucleome.org/f0f0f0f0-0000-0000-0000-aaaaaa000001;
;section;help.user-guide.getting-started.getting-started;released;https://data.4dnucleome.org/442c8aa0-dc6c-43d7-814a-854af460b011;No
2;PAGE;Getting Started;released;https://data.4dnucleome.org/d279e1c7-641c-4c4f-980d-0d1140095590;
;section;help.user-guide.getting-started.getting-started;released;https://data.4dnucleome.org/442c8aa0-dc6c-43d7-814a-854af460b011;No
2;PAGE;Data Model;released;https://data.4dnucleome.org/f0f0f0f0-0000-0000-0000-ffffff011000;
;section;help.user-guide.data-model.data-model;released;https://data.4dnucleome

;section;In situ Hi-C / Dilution Hi-C;released;https://data.4dnucleome.org/10000000-0000-0000-0000-ffff00003000;Yes
;section;DNase Hi-C / Micro-C;released;https://data.4dnucleome.org/10000000-0000-0000-0000-ffff00003001;Yes
;section;CHIA-pet / PLAC-seq;released;https://data.4dnucleome.org/10000000-0000-0000-0000-ffff00003002;Yes
;section;CHIP-seq / CUT&RUN;released;https://data.4dnucleome.org/10000000-0000-0000-0000-ffff00003003;Yes
;section;ATAC-seq;released;https://data.4dnucleome.org/10000000-0000-0000-0000-ffff00003004;Yes
;section;TrAC-loop;released;https://data.4dnucleome.org/10000000-0000-0000-0000-ffff00003005;Yes
;section;RNA-seq;released;https://data.4dnucleome.org/10000000-0000-0000-0000-ffff00003006;Yes
;section;NAD-seq;released;https://data.4dnucleome.org/10000000-0000-0000-0000-ffff00003007;Yes
;section;DAM-ID-seq;released;https://data.4dnucleome.org/10000000-0000-0000-0000-ffff00003008;Yes
;section;TSA-seq;released;https://data.4dnucleome.org/10000000-0000-0000-0000-ffff

In [29]:
text = """https://data.4dnucleome.org/10000000-0000-0000-0000-ffff00003000
https://data.4dnucleome.org/10000000-0000-0000-0000-ffff00003001
https://data.4dnucleome.org/10000000-0000-0000-0000-ffff00003002
https://data.4dnucleome.org/10000000-0000-0000-0000-ffff00003003
https://data.4dnucleome.org/10000000-0000-0000-0000-ffff00003004
https://data.4dnucleome.org/10000000-0000-0000-0000-ffff00003005
https://data.4dnucleome.org/10000000-0000-0000-0000-ffff00003006
https://data.4dnucleome.org/10000000-0000-0000-0000-ffff00003007
https://data.4dnucleome.org/10000000-0000-0000-0000-ffff00003008
https://data.4dnucleome.org/10000000-0000-0000-0000-ffff00003009
https://data.4dnucleome.org/10000000-0000-0000-0000-ffff00003010
https://data.4dnucleome.org/10000000-0000-0000-0000-ffff00003011
https://data.4dnucleome.org/10000000-0000-0000-0000-ffff00003012
https://data.4dnucleome.org/10000000-0000-0000-0000-ffff00003013
https://data.4dnucleome.org/10000000-0000-0000-0000-ffff00003014
https://data.4dnucleome.org/10000000-0000-0000-0000-ffff00003015"""

uuids = [i.split('/')[-1] for i in text.split('\n')]
print(uuids)

['10000000-0000-0000-0000-ffff00003000', '10000000-0000-0000-0000-ffff00003001', '10000000-0000-0000-0000-ffff00003002', '10000000-0000-0000-0000-ffff00003003', '10000000-0000-0000-0000-ffff00003004', '10000000-0000-0000-0000-ffff00003005', '10000000-0000-0000-0000-ffff00003006', '10000000-0000-0000-0000-ffff00003007', '10000000-0000-0000-0000-ffff00003008', '10000000-0000-0000-0000-ffff00003009', '10000000-0000-0000-0000-ffff00003010', '10000000-0000-0000-0000-ffff00003011', '10000000-0000-0000-0000-ffff00003012', '10000000-0000-0000-0000-ffff00003013', '10000000-0000-0000-0000-ffff00003014', '10000000-0000-0000-0000-ffff00003015']


In [30]:
for u in uuids:
    ff_utils.patch_metadata({"status":"draft"}, u, key = data_key)